# Extracting information from features

### What is feature engineering?

Feature engineering is the creation of new features based on existing features. The goal is to add new sets of useful information to your dataset, whether that's features for a prediction or clustering task, or features that shed insight into relationships within the dataset. Real world data is often not neat and tidy, and in addition to preprocessing steps like standardization, you'll likely have to extract and expand information that exists in the columns in your dataset. Feature engineering is something that is very dependent on the particular dataset you're analyzing, so it's very important to have an in-depth understanding of the dataset you want to model.

### Extracting features using regular expressions

Regular expressions are patterns that can be used to extract matches from text data. Here we have a string, and we want to extract the temperature digit - `45.6` - from it: 

In [ ]:
temp_string = "temperature:45.6 F"

Notice that this number is a float. We'll need a pattern to extract this float, which we can create using the `re` Python library.

In [ ]:
import re

pattern = re.compile("\d+\.\d+")

Let's break down the pattern in `re.compile`. `\d` means that we want to grab digits, and `+` means we want to grab as many as possible - so if there are two next to each other, we want both (like the 45). `\.` means we want to grab the decimal point, and then there's another `\d+` at the end to grab the digits on the right-hand side of the decimal.

To return the matching pattern, we can use `findall()`:

In [ ]:
temperature = re.findall(pattern, temp_string)

temperature

Notice that `findall()` returns a list of strings of the matched pattern. In this situation, we'd want to return the temperature as a `float`. Since we know there's only a single temperature in our string, we can do the following:

In [ ]:
temperature_num = float(re.findall(pattern, temp_string)[0])

temperature_num

This would be a little trickier in circumstances where we have multiple numerical instances in a string, but this is good enough for now. This is a very basic introduction to regular expressions; they can be a little confusing sometimes but can be extremely useful for pattern matching.

### Your turn!

Let's try this out on the `hiking` dataset. The `Length` column in the `hiking` dataset is a column of strings, but contained in the column is the mileage for the hike. We're going to extract this mileage using regular expressions, and then use a lambda in pandas to apply the extraction to the entire `DataFrame`.

First, read in the dataset and print out the `head()` of the `Length` column:

In [ ]:
import pandas as pd

dir_string = "../../datasets/"
hiking = ____(dir_string + "hiking.json")

hiking[____].____

Let's create a function that will find and return the mileage as a float value. Creating a function makes it a little easier to apply to the whole `DataFrame`. In the function, we're going to use Python's `isinstance()` to make sure what we're processing is a string—otherwise this will fail on missing values. By checking the length of our list created from `findall()`, we determine if there are any matches, and if so, we return them.

Fill in the correct `re` functions and pattern below:

In [ ]:
def return_mileage(length):
    if isinstance(length, str):
        pattern = ____(r"____")
        mile = ____(____, length)
        if len(mile) == 1:
            return float(mile[0])
    else:
        return 0

Next, let's use `apply()` on the `Length` column to generate a new column with just the hiking mileage. Pass the `return_mileage` function into apply, after the `lambda`:

In [ ]:
hiking["Length Mileage"] = hiking[____].____(lambda row: ____(row))

Finally, let's compare our new column to the original column to make sure it worked:

In [ ]:
print(hiking[["Length", "Length Mileage"]].head())
print("\n")
print(hiking[["Length", "Length Mileage"]].dtypes)

### Encoding variables: binary

Because models in scikit-learn require numerical input, if your dataset contains categorical variables, you'll have to encode them. Encoding binary values is actually quite simple, and can be done in both pandas and scikit-learn. You might want encode variables in pandas if you're not finished preprocessing, or if you're interested in further exploratory work once you've encoded. On the other hand, you may want to use scikit-learn if, for example, you're implementing encoding as part of scikit-learn's pipeline functionality, which allows you to string different parts of the machine learning process together. 
 
Let's look at an example using the `hiking` dataset. One column that needs encoding is the `Accessible` column, which has values of either `Y` or `N`.

In [ ]:
hiking["Accessible"].head()

In pandas, we can use `apply()` to encode 1s and 0s in a `DataFrame` column, using a simple conditional that returns a 1 if the value in `Accessible` is `Y`, and a 0 if the value is `N`.

In [ ]:
hiking["Accessible_enc"] = hiking["Accessible"].____(lambda val: 1 if val == ____ else ____)

hiking[["Accessible", "Accessible_enc"]].head()

Looking at a side by side comparison of the columns, you can see that the column is now numerically encoded.

You can also do this in scikit-learn using `LabelEncoder`. Creating a `LabelEncoder` object also allows you to reuse this encoding on other data, such as on new data or a test set. You can use `fit_transform()` to both fit the encoder to the data as well as transform the column. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = ____
hiking["Accessible_enc_le"] = ____(hiking[____])

hiking[["Accessible", "Accessible_enc", "Accessible_enc_le"]].head()

Printing out `Accessible` and its encoded counterparts, we can see that the `Y` and `N` values have been encoded to 1s and 0s in the same manner in both pandas and scikit-learn.

### Encoding variables: one-hot

One-hot encoding encodes categorical variables into 1s and 0s when you have more than two variables to encode. It works by looking at the entire list of unique values in a column, transforming each value into an array, and designating a 1 in the appropriate position to encode that a particular value occurs.

Let's look at a toy example to see how this works, taking a small dataset of colors:

In [ ]:
color_ex = pd.Series(["blue", "red", "green", "red"])

Here we have three values: blue, green, and red. In order to one-hot encode these values, we can use `get_dummies()` to generate columns for each.

In [ ]:
color_enc = pd.____(color_ex)
color_enc

To concatenate these columns back to the original data, you can use `pd.concat`:

In [ ]:
____([color_ex, color_enc], axis=1)

Blue has a one in the first position followed by two zeros, green has a one in the second position, and red has a one in the last position. So the encoded colors look like this, where a value of `1` indicates that that color appeared in the original column in that particular row.

### Your turn!

One of the columns in the `volunteer` dataset, `category_desc`, gives category descriptions for the volunteer opportunities listed. Because it is a categorical variable with more than two categories, let's practice using one-hot encoding to transform this column numerically. First read in the dataset, and then use `get_dummies()` on the `category_desc` column.

In [ ]:
volunteer = ____(dir_string + "volunteer.csv")

category_enc = ____(____)
category_enc.head()

Finally, we can concatenate `category_enc` back onto the `volunteer` DataFrame and take a look at a few rows and their encodings:

In [ ]:
volunteer_enc = ____([volunteer, ____], axis=____)

volunteer_enc[["category_desc", "Strengthening Communities", "Helping Neighbors in Need"]].tail()

### Aggregate stats

If you had, say, a collection of features related to a single feature, like temperature or running time, you might want to take an average or median to use as a feature for modeling instead. 

A common method of feature engineering is to take an aggregate of a set of numbers to use in place of those features. This can be helpful in reducing the dimensionality of your feature space, or perhaps you simply don't need multiple similar values that are close in distance to each other. 

Let's say we have a `DataFrame` of running times named `running_times_5k`:

In [ ]:
running_dict = {"name": ["Sue", "Mark", "Sean", "Erin", "Jenny", "Russell"], 
                "run1": [20.1, 16.5, 23.5, 21.7, 25.8, 30.9], 
                "run2": [18.5, 17.1, 25.1, 21.1, 27.1, 29.6], 
                "run3": [19.6, 16.9, 25.2, 20.9, 26.1, 31.4], 
                "run4": [20.3, 17.6, 24.6, 22.1, 26.7, 30.4], 
                "run5": [18.3, 17.3, 23.9, 22.2, 26.9, 29.9]}

running_times_5k = pd.DataFrame(running_dict)
running_times_5k

Instead of using each individual run time to build our model, let's use the mean of these five runs for each person in the dataset.

To generate the `mean` column, we can apply the `mean()` function to the DataFrame. We need to specifiy `axis=1` in order to get our column.

In [ ]:
running_times_5k["mean"] = running_times_5k.____(____)

running_times_5k

And now we have an aggregate mean column.

### Your turn!

Using the `running_times_5k` dataset from above, let's generate some additional aggregate stats.

Let's create a column for the `median()`, the total amount of time for each runner using `sum()`, and let's capture each runner's best time using `min()`.

In [ ]:
running_times_5k["median"] = ____
running_times_5k["total"] = ____
running_times_5k["fastest_time"] = ____

Let's take a look at the four columns we generated:

In [ ]:
running_times_5k[["mean", "median", "total", "fastest_time"]]